In [4]:
import pandas as pd
import glob
import re

# Define the path to the folder containing the CSV files
folder_path = 'Data/POS'

# Use glob to get all CSV files in the folder
csv_files = glob.glob(f'{folder_path}/*.csv')

# Initialize an empty list to store the DataFrames
df_list = []

# Define a regular expression pattern to extract week information from filenames
pattern = re.compile(r'POS__(\d{4}-\d{2}-\d{2})')

# Loop over the list of files and read each file into a DataFrame
for file in csv_files:
    # Extract the date_start and date_end from the filename
    match = pattern.search(file)
    if match:
        week_start = match.groups()[0]

        df = pd.read_csv(file)

        df['date'] = week_start
        df['date'] = pd.to_datetime(df['date'])
        df['year'] = df['date'].dt.year
        df['week'] = df['date'].dt.isocalendar().week
        df['yearly_week'] = df['year'].astype(str) + '-W' + df['week'].astype(str).str.zfill(2)

        df_list.append(df)

# Concatenate all DataFrames in the list into a single DataFrame
combined_df = pd.concat(df_list, ignore_index=True)

# Display the combined DataFrame
combined_df

,gerecht,prijs_€,aantal_bestellingen,laatst_besteld,date,year,week,yearly_week,Something went wrong while creating the file
0,Frikandel speciaal,3.00,102.0,2021-07-08 19:39:04,2021-07-01,2021,26,2021-W26,NaN
1,Frikandel,1.80,79.0,2021-07-08 20:19:25,2021-07-01,2021,26,2021-W26,NaN
2,Friet,2.00,77.0,2021-07-08 20:19:25,2021-07-01,2021,26,2021-W26,NaN
3,Friet speciaal,3.10,76.0,2021-07-08 19:34:12,2021-07-01,2021,26,2021-W26,NaN
4,Kroket,1.80,56.0,2021-07-08 20:01:37,2021-07-01,2021,26,2021-W26,NaN
...,...,...,...,...,...,...,...,...,...
16650,Satékroket,2.35,1.0,2024-04-26 19:00:59,2024-04-25,2024,17,2024-W17,NaN
16651,Shooter,3.40,1.0,2024-05-02 19:43:46,2024-04-25,2024,17,2024-W17,NaN
16652,Spa Blauw,2.50,1.0,2024-05-01 17:17:09,2024-04-25,2024,17,2024-W17,NaN
16653,Spicy Chicken Burger,6.30,1.0,2024-04-27 16:38:34,2024-04-25,2024,17,2024-W17,NaN


In [2]:
combined_df

,gerecht,prijs_€,aantal_bestellingen,laatst_besteld,week,Something went wrong while creating the file
0,Frikandel speciaal,3.00,102.0,2021-07-08 19:39:04,2021-07-01,NaN
1,Frikandel,1.80,79.0,2021-07-08 20:19:25,2021-07-01,NaN
2,Friet,2.00,77.0,2021-07-08 20:19:25,2021-07-01,NaN
3,Friet speciaal,3.10,76.0,2021-07-08 19:34:12,2021-07-01,NaN
4,Kroket,1.80,56.0,2021-07-08 20:01:37,2021-07-01,NaN
...,...,...,...,...,...,...
16650,Satékroket,2.35,1.0,2024-04-26 19:00:59,2024-04-25,NaN
16651,Shooter,3.40,1.0,2024-05-02 19:43:46,2024-04-25,NaN
16652,Spa Blauw,2.50,1.0,2024-05-01 17:17:09,2024-04-25,NaN
16653,Spicy Chicken Burger,6.30,1.0,2024-04-27 16:38:34,2024-04-25,NaN
